## Check Python Version of Environment

In [ ]:
from platform import python_version

print(python_version())

## 1. Install & Import Dependencies

### 1.1 Install Pytorch

In [ ]:
''' Visit "https://pytorch.org/get-started/locally/" 
to download compatible version of pytorch '''

#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

### 1.2 Setup YOLO V9

#### 1.2.1 Clone YOLO V9 Repository

In [ ]:
import os

if not os.path.exists('yolov9'):
    !git clone https://github.com/SkalskiP/yolov9.git
    !cd yolov9 & pip install -r requirements.txt

#### 1.2.2 Download Weights

In [ ]:

import urllib.request

urls = [
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt",
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt",
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt",
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt"
]

directory = "yolov9/weights"

os.makedirs(directory, exist_ok=True)

download_count = 0
for url in urls:
    filename = url.split("/")[-1] 
    filepath = f"{directory}/{filename}" 
    
    # Check if the file already exists
    if not os.path.exists(filepath):
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, filepath)
        download_count += 1
    else:
        print(f"{filename} already exists. Skipping download.")
        
print(f"Download Complete. Downloaded {download_count} files")

### 1.3 Import Dependencies

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import glob 
import random
import os

#### 1.4 Check for GPU

In [ ]:
# Check PyTorch GPU availability
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Is Available: {torch.cuda.is_available()}")
else:
    print("GPU is not available.")

## 2. Detect Using Pretrained Weights

In [ ]:
img = 'dataset/test/images/33.png'

### 2.1 Using Gelan-C

In [ ]:
!cd yolov9 && python detect.py \
--img 1000 --conf 0.1 --device 0 \
--weights {HOME}/weights/gelan-c.pt \
--source dataset/test/images/23.png

## Plot Sample Images

In [ ]:
# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

In [ ]:
def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # Denormalize the coordinates.
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)

        thickness = max(2, int(w/275))
                
        cv2.rectangle(
            image, 
            (xmin, ymin), (xmax, ymax),
            color=(0, 0, 255),
            thickness=thickness
        )
    return image

In [ ]:
def plot(image_paths, label_paths, name):
    plt.figure(figsize=(15, 12))
    image_name = f"dataset/test/images\\{name}"
    image = cv2.imread(image_name)
    label_name = name[:-3]+'txt'
    with open(os.path.join(label_paths, label_name), 'r') as f:
        bboxes = []
        labels = []
        label_lines = f.readlines()
        for label_line in label_lines:
            label = label_line[0]
            bbox_string = label_line[2:]
            x_c, y_c, w, h = bbox_string.split(' ')
            x_c = float(x_c)
            y_c = float(y_c)
            w = float(w)
            h = float(h)
            bboxes.append([x_c, y_c, w, h])
            labels.append(label)
    result_image = plot_box(image, bboxes, labels)
    plt.subplot(2, 2, 1)
    plt.imshow(result_image[:, :, ::-1])
    plt.axis('off')
    plt.subplots_adjust(wspace=1)
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize a few training images.
plot(
    image_paths='dataset/test/images/', 
    label_paths='dataset/test/labels/',
    name='23.png',
)

## Train Model on Custom Data

In [ ]:
EPOCHS = 20
BATCH = 4
IMG_SIZE = 1000

In [ ]:
!cd yolov9

In [ ]:
# !cd yolov9 && python train_dual.py \
# --workers 8 \
# --batch 4  \
# --img 600 \
# --epochs 5 \
# --data data.yaml \
# --weights weights/yolov9-c.pt \
# --device 0 \
# --cfg models/detect/yolov9_custom.yaml \
# --hyp data/hyps/hyp.scratch-high.yaml

## Test Custom Model

In [ ]:
python val.py --img 1000 --batch 8 --conf 0.001 --iou 0.7 --device 0 --data data.yaml --weights runs/train/exp12/weights/best.pt

In [ ]:
!cd yolov9 && python detect.py \
--img 1000 --conf 0.1 --device 0 \
--weights runs/train/exp12/weights/best.pt \
--source dataset/test/images/23.png

In [ ]:
from IPython.display import Image
Image(filename="yolov9/runs/detect/exp/23.png", width=600)
 